In [0]:
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)

In [0]:
class ResidualBlock(nn.Module):
    """
    The residual block used by ResNet.
    
    Args:
        in_channels: The number of channels (feature maps) of the incoming embedding
        out_channels: The number of channels after the first convolution
        stride: Stride size of the first convolution, used for downsampling
    """
    
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()        
        if stride > 1 or in_channels != out_channels:
            # Add strides in the skip connection and zeros for the new channels.
            self.skip = Lambda(lambda x: F.pad(x[:, :, ::stride, ::stride],
                                               (0, 0, 0, 0, 0, out_channels - in_channels),
                                               mode="constant", value=0))
        else:
            self.skip = nn.Sequential()
            
        # TODO: Initialize the required layers
        self.layer = nn.Sequential(
                nn.Conv2d(in_channels,out_channels,kernel_size=3,stride=stride,padding=1,bias=False),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_channels,out_channels,kernel_size=3,stride=1,padding=1,bias=False),
                nn.BatchNorm2d(out_channels)
            )
    def forward(self, inputs):
        # TODO: Execute the required layers and functions
        out = self.layer(inputs)
        out += self.skip(inputs)
        out = F.relu(out)
        return out

In [0]:
class ResidualStack(nn.Module):
    """
    A stack of residual blocks.
    
    Args:
        in_channels: The number of channels (feature maps) of the incoming embedding
        out_channels: The number of channels after the first layer
        stride: Stride size of the first layer, used for downsampling
        num_blocks: Number of residual blocks
    """
    
    def __init__(self, in_channels, out_channels, stride, num_blocks):
        super().__init__()
        
        # TODO: Initialize the required layers (blocks)
        layer = []
        layer.append(ResidualBlock(in_channels, out_channels, stride))
        for i in range(1,num_blocks):
            layer.append(ResidualBlock(out_channels, out_channels))
        self.layer = nn.Sequential(*layer)

    def forward(self, out):
        # TODO: Execute the layers (blocks)
        out = self.layer(out)
        return out

In [0]:
n = 5
num_classes = 10

# TODO: Implement ResNet via nn.Sequential
resnet = nn.Sequential(
    nn.Conv2d(3,16,3,1,1,bias=False),
    nn.BatchNorm2d(16),
    nn.ReLU(inplace=True),
    ResidualStack(16,16,1,n),
    ResidualStack(16,32,2,n),
    ResidualStack(32,64,2,n),
    nn.AvgPool2d(8),
    Lambda(lambda x: torch.squeeze(x)),
    nn.Linear(64,num_classes),
    # nn.Softmax(dim=1) 
)

In [0]:
def initialize_weight(module):
    if isinstance(module, (nn.Linear, nn.Conv2d)):
        nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
    elif isinstance(module, nn.BatchNorm2d):
        nn.init.constant_(module.weight, 1)
        nn.init.constant_(module.bias, 0)
        
resnet.apply(initialize_weight);

In [0]:
class CIFAR10Subset(torchvision.datasets.CIFAR10):
    """
    Get a subset of the CIFAR10 dataset, according to the passed indices.
    """
    def __init__(self, *args, idx=None, **kwargs):
        super().__init__(*args, **kwargs)
        
        if idx is None:
            return
        
        self.data = self.data[idx]
        targets_np = np.array(self.targets)
        self.targets = targets_np[idx].tolist()

In [0]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, 4),
    transforms.ToTensor(),
    normalize,
])
transform_eval = transforms.Compose([
    transforms.ToTensor(),
    normalize
])

In [10]:
ntrain = 45_000
train_set = CIFAR10Subset(root='./data', train=True, idx=range(ntrain),
                          download=True, transform=transform_train)
val_set = CIFAR10Subset(root='./data', train=True, idx=range(ntrain, 50_000),
                        download=True, transform=transform_eval)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform_eval)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified


In [0]:
dataloaders = {}
dataloaders['train'] = torch.utils.data.DataLoader(train_set, batch_size=128,
                                                   shuffle=True, num_workers=2,
                                                   pin_memory=True)
dataloaders['val'] = torch.utils.data.DataLoader(val_set, batch_size=128,
                                                 shuffle=False, num_workers=2,
                                                 pin_memory=True)
dataloaders['test'] = torch.utils.data.DataLoader(test_set, batch_size=128,
                                                  shuffle=False, num_workers=2,
                                                  pin_memory=True)

In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
resnet.to(device);

In [0]:
def run_epoch(model, optimizer, dataloader, train):
    """
    Run one epoch of training or evaluation.
    
    Args:
        model: The model used for prediction
        optimizer: Optimization algorithm for the model
        dataloader: Dataloader providing the data to run our model on
        train: Whether this epoch is used for training or evaluation
        
    Returns:
        Loss and accuracy in this epoch.
    """
    # TODO: Change the necessary parts to work correctly during evaluation (train=False)
    device = next(model.parameters()).device

    epoch_loss = 0.0
    epoch_acc = 0.0

    # Set model to training mode (for e.g. batch normalization, dropout)
    if train:
        model.train()
    else:
        model.eval()

    for xb, yb in dataloader:
        xb, yb = xb.to(device), yb.to(device)
      # zero the parameter gradients
        if train:
            optimizer.zero_grad()

      # forward
        with torch.set_grad_enabled(True):
            pred = model(xb)
            loss = F.cross_entropy(pred, yb)
            top1 = torch.argmax(pred, dim=1)
            ncorrect = torch.sum(top1 == yb)

        if train:  
            loss.backward()
            optimizer.step()

      # statistics
        epoch_loss += loss.item()
        epoch_acc += ncorrect.item()

    epoch_loss /= len(dataloader.dataset)
    epoch_acc /= len(dataloader.dataset)
    return epoch_loss, epoch_acc

In [0]:
def fit(model, optimizer, lr_scheduler, dataloaders, max_epochs, patience):
    """
    Fit the given model on the dataset.
    
    Args:
        model: The model used for prediction
        optimizer: Optimization algorithm for the model
        lr_scheduler: Learning rate scheduler that improves training
                      in late epochs with learning rate decay
        dataloaders: Dataloaders for training and validation
        max_epochs: Maximum number of epochs for training
        patience: Number of epochs to wait with early stopping the
                  training if validation loss has decreased
                  
    Returns:
        Loss and accuracy in this epoch.
    """
    
    best_acc = 0
    curr_patience = 0
    PATH = './data'
    for epoch in range(max_epochs):
        train_loss, train_acc = run_epoch(model, optimizer, dataloaders['train'], train=True)
        lr_scheduler.step()
        print(f"Epoch {epoch + 1: >3}/{max_epochs}, train loss: {train_loss:.2e}, accuracy: {train_acc * 100:.2f}%")

        val_loss, val_acc = run_epoch(model, None, dataloaders['val'], train=False)
        print(f"Epoch {epoch + 1: >3}/{max_epochs}, val loss: {val_loss:.2e}, accuracy: {val_acc * 100:.2f}%")

        # TODO: Add early stopping and save the best weights (in best_model_weights)
        if val_acc > best_acc:
            curr_patience = 0
            best_acc = val_acc
            best_model_weights = copy.deepcopy(model.state_dict())
        else:
            curr_patience += 1
        if curr_patience >= patience:
            break
    
    model.load_state_dict(best_model_weights)

In [15]:
optimizer = torch.optim.SGD(resnet.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100, 150], gamma=0.1)

# Fit model
fit(resnet, optimizer, lr_scheduler, dataloaders, max_epochs=200, patience=50)

Epoch   1/200, train loss: 1.41e-02, accuracy: 34.59%
Epoch   1/200, val loss: 1.31e-02, accuracy: 39.54%
Epoch   2/200, train loss: 9.69e-03, accuracy: 54.78%
Epoch   2/200, val loss: 9.48e-03, accuracy: 58.80%
Epoch   3/200, train loss: 7.65e-03, accuracy: 65.06%
Epoch   3/200, val loss: 6.87e-03, accuracy: 70.58%
Epoch   4/200, train loss: 6.33e-03, accuracy: 71.52%
Epoch   4/200, val loss: 6.77e-03, accuracy: 71.40%
Epoch   5/200, train loss: 5.53e-03, accuracy: 75.38%
Epoch   5/200, val loss: 5.65e-03, accuracy: 76.20%
Epoch   6/200, train loss: 5.00e-03, accuracy: 77.77%
Epoch   6/200, val loss: 6.95e-03, accuracy: 72.02%
Epoch   7/200, train loss: 4.62e-03, accuracy: 79.42%
Epoch   7/200, val loss: 5.40e-03, accuracy: 77.36%
Epoch   8/200, train loss: 4.27e-03, accuracy: 81.02%
Epoch   8/200, val loss: 5.27e-03, accuracy: 79.20%
Epoch   9/200, train loss: 4.00e-03, accuracy: 82.33%
Epoch   9/200, val loss: 4.46e-03, accuracy: 82.18%
Epoch  10/200, train loss: 3.82e-03, accuracy:

In [16]:
test_loss, test_acc = run_epoch(resnet, None, dataloaders['test'], train=False)
print(f"Test loss: {test_loss:.1e}, accuracy: {test_acc * 100:.2f}%")

Test loss: 2.9e-03, accuracy: 92.20%
